In [19]:
import os

import polars as pl

os.chdir("../../../")

# predictables\encoding\src\lagged_mean_encoding\_dynamic_rolling_sum.py
from predictables.encoding.src.lagged_mean_encoding._dynamic_rolling_sum import (
    _formatted_category_cols,
    _formatted_date_col,
    _get_date_map,
    _get_original_order,
    _group_by_categories,
    _group_by_no_categories,
    _reversed_date_col,
    _rolling_sum_categories,
    _rolling_sum_no_categories,
)

In [23]:
date_fmt_str = "%m/%d/%Y"

lf = pl.scan_csv(
    "/home/aweaver/work/predictables/predictables/encoding/tests/rolling_date_example.csv"
).with_columns(
    [
        pl.col("date").str.to_date(format=date_fmt_str).cast(pl.Date).name.keep(),
        pl.col("incr_value").cast(pl.Float64).name.keep(),
        pl.col("month_prior")
        .str.to_date(format=date_fmt_str)
        .cast(pl.Date)
        .name.keep(),
        pl.col("year_prior").str.to_date(format=date_fmt_str).cast(pl.Date).name.keep(),
        pl.col("rolling_sum").cast(pl.Float64).name.keep(),
        pl.col("date").str.to_date(format=date_fmt_str).cast(pl.Date).alias("cur_date"),
        pl.col("cat1").cast(pl.Utf8).cast(pl.Categorical).name.keep(),
        pl.col("cat2").cast(pl.Utf8).cast(pl.Categorical).name.keep(),
    ]
)
lf.head().collect()

date,incr_value,month_prior,year_prior,rolling_sum,cat1,cat2,cur_date
date,f64,date,date,f64,cat,cat,date
2014-01-01,1.0,2013-12-01,2012-12-01,0.0,"""B""","""C""",2014-01-01
2014-01-02,1.0,2013-12-02,2012-12-02,0.0,"""A""","""C""",2014-01-02
2014-01-03,1.0,2013-12-03,2012-12-03,0.0,"""B""","""D""",2014-01-03
2014-01-04,1.0,2013-12-04,2012-12-04,0.0,"""B""","""D""",2014-01-04
2014-01-05,1.0,2013-12-05,2012-12-05,0.0,"""B""","""E""",2014-01-05


In [26]:
_rolling_sum_no_categories(lf, "incr_value", "date", "incr").collect()

dcol: date
dcol type: <class 'str'>
column type: [Date]
dcol: date
dcol type: <class 'str'>
column type: [Date]


thread '<unnamed>' panicked at /home/runner/work/polars/polars/crates/polars-lazy/src/frame/mod.rs:1019:18:
called `Result::unwrap()` on an `Err` value: ColumnNotFound(ErrString("date"))
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: called `Result::unwrap()` on an `Err` value: ColumnNotFound(ErrString("date"))